# Vulnerability Scanner (Binder Compatible)

In [ ]:
import pandas as pd
import numpy as np
import joblib
import ipywidgets as widgets
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# === Module 1: Administration ===
class AdminModule:
    def __init__(self):
        self.settings = {"min_confidence": 0.7, "max_scan_depth": 10}
        self.users = {"admin": "password123"}

admin = AdminModule()
display(widgets.HTML("<h3>Admin Module Loaded</h3>"))

# === Module 2: Training ===
class TrainingModule:
    def __init__(self):
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)

    def train(self, data):
        X = data.drop(columns=["vulnerability"])
        y = data["vulnerability"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)
        predictions = self.model.predict(X_test)
        acc = accuracy_score(y_test, predictions)
        joblib.dump(self.model, "vuln_model.pkl")
        return f"Training completed. Model accuracy: {acc:.2f}"

data = pd.DataFrame({
    "code_length": np.random.randint(50, 500, 100),
    "complexity": np.random.randint(1, 10, 100),
    "has_sql_injection": np.random.randint(0, 2, 100),
    "vulnerability": np.random.randint(0, 2, 100)
})

trainer = TrainingModule()
result = trainer.train(data)
display(widgets.HTML(f"<h3>{result}</h3>"))

# === Module 3: Execution ===
class ExecutionModule:
    def __init__(self):
        self.model = joblib.load("vuln_model.pkl")

    def analyze_code(self, features):
        prediction = self.model.predict([features])
        return "Vulnerability detected!" if prediction[0] == 1 else "Code is secure."

executor = ExecutionModule()
display(widgets.HTML("<h3>Execution Module Loaded</h3>"))

# === Module 4: Code Analysis with Interactive Interface ===
def analyze_code_interface():
    code_length = widgets.IntSlider(min=50, max=500, step=10, description="Code Length")
    complexity = widgets.IntSlider(min=1, max=10, step=1, description="Complexity")
    has_sql_injection = widgets.Dropdown(options=[(0, "No"), (1, "Yes")], description="SQL Injection")

    def on_click(b):
        result = executor.analyze_code([code_length.value, complexity.value, has_sql_injection.value])
        display(widgets.HTML(f"<h3>{result}</h3>"))

    button = widgets.Button(description="Analyze Code")
    button.on_click(on_click)

    display(code_length, complexity, has_sql_injection, button)

analyze_code_interface()